In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader,random_split
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Subset, WeightedRandomSampler

from torchvision import transforms, datasets

from sklearn.metrics import (
    roc_curve,
    auc,
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score
)
from sklearn.utils import class_weight

from PIL import Image
from torch.optim.lr_scheduler import StepLR
import random
import time
from tqdm import tqdm
from collections import Counter
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
seed = 42
torch.manual_seed(seed)           
random.seed(seed)                  
np.random.seed(seed)               
torch.cuda.manual_seed(seed)       
torch.backends.cudnn.deterministic = True  
torch.backends.cudnn.benchmark = False

# **TENSORBOARD SETUP**

In [ ]:
pip install tensorboard

**Note: change the name of your directory. For example, if your model's name is "Deit" then log_dir="runs/deit". *Do not change the main directory "runs"!***

In [ ]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter(log_dir="runs/ConvNextv2") 

# **DATA LOADING & PREPROCESSING**

In [ ]:
train_data_path = '/kaggle/input/train'
test_data_path = '/kaggle/input/test'

In [ ]:
# Load dataset
train_dataset = datasets.ImageFolder(root=train_data_path)

# Print class-to-index mapping
print("Class-to-Index Mapping:", train_dataset.class_to_idx)

# Get counts of each label
label_counts = Counter([sample[1] for sample in train_dataset.samples])

# Print counts for each label
print("Label Counts:")
for label, count in label_counts.items():
    class_name = list(train_dataset.class_to_idx.keys())[list(train_dataset.class_to_idx.values()).index(label)]
    print(f"{class_name}: {count} images")


In [ ]:
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize
    transforms.RandomHorizontalFlip(p=0.5),  # Horizontal flip
    transforms.RandomVerticalFlip(p=0.5),  # Vertical flip
    transforms.RandomRotation(degrees=10),  # Rotation
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),  # Color jitter
    transforms.GaussianBlur(kernel_size=(3, 3), sigma=(0.1, 0.5)),  # Gaussian noise
    transforms.ToTensor(),  # Convert to Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize
    transforms.ToTensor(),  # Convert to Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])

In [ ]:
train_dataset = datasets.ImageFolder(root=train_data_path , transform=train_transforms)
test_dataset = datasets.ImageFolder(root=test_data_path, transform=test_transforms)

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

print(f"Training Set Size: {len(train_dataset)} images")
print(f"Testing Set Size: {len(test_dataset)} images")

In [ ]:
train_labels = [label for _, label in train_dataset.samples]
train_label_counts = Counter(train_labels)

test_labels = [label for _, label in test_dataset.samples]
test_label_counts = Counter(test_labels)

class_names = train_dataset.classes
train_label_names = {class_names[label]: count for label, count in train_label_counts.items()}
test_label_names = {class_names[label]: count for label, count in test_label_counts.items()}

# Visualize label distributions
print("Training Label Distribution:", train_label_names)
print("Testing Label Distribution:", test_label_names)

def plot_label_distribution(label_counts, title):
    labels, counts = zip(*label_counts.items())
    plt.figure(figsize=(4, 2))
    plt.bar(labels, counts)
    plt.xlabel('Class Labels')
    plt.ylabel('Number of Samples')
    plt.title(title)
    plt.xticks(rotation=45)
    plt.show()

plot_label_distribution(train_label_names, "Training Dataset Label Distribution")
plot_label_distribution(test_label_names, "Testing Dataset Label Distribution")

# **LOAD MODEL**

**Note: Load your model here**

In [ ]:
from timm import create_model
import torch.nn as nn

# Load the ConvNeXtV2 model
model = create_model("convnextv2_tiny.fb_in1k", pretrained=True)

# Modify the classification head for binary classification (2 classes)
num_features = model.head.fc.in_features  # Access the input size of the fully connected layer
model.head.fc = nn.Linear(num_features, 2)  # Replace with a new fully connected layer

# Send model to GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)

In [ ]:
labels = [label for _, label in train_dataset.samples] 
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(labels), y=labels)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.1, verbose=True)

# **TRAINING**

In [ ]:
class EarlyStopping:
    def __init__(self, patience=5, delta=0.0):
        self.patience = patience
        self.delta = delta
        self.best_loss = float('inf')
        self.counter = 0
        self.early_stop = False

    def __call__(self, train_loss):
        if train_loss < self.best_loss - self.delta:
            self.best_loss = train_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                print(f"Early stopping triggered after {self.patience} epochs with no improvement.")
                self.early_stop = True

In [ ]:
def plot_training_results(train_losses, train_accuracies, learning_rates, fps, gpu_usage):
    # Learning Rate
    plt.figure(figsize=(8, 6))
    plt.plot(range(1, len(learning_rates) + 1), learning_rates, marker='o')
    plt.xlabel("Epochs")
    plt.ylabel("Learning Rate")
    plt.title("Learning Rate Schedule")
    plt.grid()
    plt.show()

    # Training Accuracy
    plt.figure(figsize=(8, 6))
    plt.plot(range(1, len(train_accuracies) + 1), train_accuracies, label="Training Accuracy", marker='o')
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy (%)")
    plt.title("Training Accuracy")
    plt.legend()
    plt.grid()
    plt.show()

    # FPS
    plt.figure(figsize=(8, 6))
    plt.plot(range(1, len(fps) + 1), fps, marker='o')
    plt.xlabel("Epochs")
    plt.ylabel("Frames Per Second (FPS)")
    plt.title("Training Speed (FPS)")
    plt.grid()
    plt.show()

    # GPU Usage
    allocated, reserved = zip(*gpu_usage)
    plt.figure(figsize=(8, 6))
    plt.plot(range(len(allocated)), allocated, label="GPU Allocated (GB)")
    plt.plot(range(len(reserved)), reserved, label="GPU Reserved (GB)")
    plt.xlabel("Batches")
    plt.ylabel("GPU Memory (GB)")
    plt.title("GPU Memory Usage")
    plt.legend()
    plt.grid()
    plt.show()

def write_training_results(train_losses, train_accuracies, learning_rates, fps, gpu_usage):
    # Learning Rate
    plt.figure(figsize=(8, 6))
    plt.plot(range(1, len(learning_rates) + 1), learning_rates, marker='o')
    plt.xlabel("Epochs")
    plt.ylabel("Learning Rate")
    plt.title("Learning Rate Schedule")
    plt.grid()
    fig = plt.gcf()
    if writer:
        writer.add_figure("Learning Rate Schedule", fig)

    # Training Accuracy
    plt.figure(figsize=(8, 6))
    plt.plot(range(1, len(train_accuracies) + 1), train_accuracies, label="Training Accuracy", marker='o')
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy (%)")
    plt.title("Training Accuracy")
    plt.legend()
    plt.grid()
    fig = plt.gcf()
    if writer:
        writer.add_figure("Training Accuracy", plt.gcf())

    # FPS
    plt.figure(figsize=(8, 6))
    plt.plot(range(1, len(fps) + 1), fps, marker='o')
    plt.xlabel("Epochs")
    plt.ylabel("Frames Per Second (FPS)")
    plt.title("Training Speed (FPS)")
    plt.grid()
    fig = plt.gcf()
    if writer:
        writer.add_figure("Training Speed (FPS)", fig)

    # GPU Usage
    allocated, reserved = zip(*gpu_usage)
    plt.figure(figsize=(8, 6))
    plt.plot(range(len(allocated)), allocated, label="GPU Allocated (GB)")
    plt.plot(range(len(reserved)), reserved, label="GPU Reserved (GB)")
    plt.xlabel("Batches")
    plt.ylabel("GPU Memory (GB)")
    plt.title("GPU Memory Usage")
    plt.legend()
    plt.grid()
    fig = plt.gcf()
    if writer:
        writer.add_figure("GPU Memory Usage", fig)

In [ ]:
def train_model(model, train_loader, optimizer, scheduler, criterion, device, patience=5, num_epochs=50):
    early_stopping = EarlyStopping(patience=patience)
    model.to(device)

    # Lists to track metrics
    train_losses, train_accuracies, learning_rates = [], [], []
    gpu_usage, fps = [], []

    for epoch in range(num_epochs):
        # Fetch and print the current learning rate
        current_lr = optimizer.param_groups[0]['lr']
        learning_rates.append(current_lr)  # Track learning rate
        writer.add_scalar("Learning Rate", current_lr, epoch)
        
        print(f"\nEpoch {epoch+1}/{num_epochs}, Learning Rate: {current_lr:.6f}")

        model.train()
        train_loss, correct, total = 0.0, 0, 0

        # GPU memory tracking
        start_time = time.time()
        for images, labels in tqdm(train_loader, desc="Training Batches"):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()

            # Forward pass
            outputs = model(images)

            # Check for logits attribute (e.g., in DeiT models)
            if hasattr(outputs, 'logits'):
                outputs = outputs.logits

            # Compute loss and backpropagate
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        # Monitor GPU memory usage
        gpu_allocated = torch.cuda.memory_allocated(device) / (1024 ** 3)  # Convert to GB
        gpu_reserved = torch.cuda.memory_reserved(device) / (1024 ** 3)   
        gpu_usage.append((gpu_allocated, gpu_reserved))

        epoch_loss = train_loss / len(train_loader)
        epoch_acc = 100. * correct / total
        train_losses.append(epoch_loss)  # Track training loss
        train_accuracies.append(epoch_acc)  # Track training accuracy

        # Log training loss and accuracy to TensorBoard
        writer.add_scalar("Loss/Train", epoch_loss, epoch)
        writer.add_scalar("Accuracy/Train", epoch_acc, epoch)

        # Measure FPS
        end_time = time.time()
        fps_epoch = len(train_loader.dataset) / (end_time - start_time)
        fps.append(fps_epoch)

        print(f"Training Loss: {epoch_loss:.4f}, Training Accuracy: {epoch_acc:.2f}%, FPS: {fps_epoch:.2f}")
        print(f"GPU Memory Allocated: {gpu_allocated:.2f} GB, GPU Memory Reserved: {gpu_reserved:.2f} GB")

        # Log FPS
        writer.add_scalar("Training/FPS", fps_epoch, epoch)

        # Step the LR scheduler
        scheduler.step(epoch_loss)

        # Early Stopping Check
        early_stopping(epoch_loss)
        if early_stopping.early_stop:
            print("Stopping early due to convergence.")
            break

    # Plot Results
    plot_training_results(train_losses, train_accuracies, learning_rates, fps, gpu_usage)
    write_training_results(train_losses, train_accuracies, learning_rates, fps, gpu_usage)

In [ ]:
train_model(
    model=model,
    train_loader=train_loader,
    optimizer=optimizer,
    scheduler=scheduler,
    criterion=criterion,
    device=device,
    patience=5,  
    num_epochs=50
)

# **EVALUATION**

In [ ]:
def evaluate_model_with_metrics(model, loader, binary_classification=True):
    model.eval()
    all_preds = []
    all_labels = []
    all_probs = []  
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to('cuda'), labels.to('cuda')
            
            # Get model outputs
            outputs = model(images)
            
            # For DeiT models, extract logits
            if hasattr(outputs, 'logits'):  
                outputs = outputs.logits
            
            # Get predicted classes
            _, predicted = outputs.max(1)

            # Accumulate metrics
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            # Probabilities for ROC AUC
            if binary_classification:
                probabilities = F.softmax(outputs, dim=1)[:, 1].cpu().numpy()
                all_probs.extend(probabilities)

    # Accuracy
    accuracy = 100. * correct / total
    print(f"Accuracy on test set: {accuracy:.2f}%")
    writer.add_scalar("Evaluation/Accuracy", accuracy)

    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)

    # Confusion Matrix
    cm = confusion_matrix(all_labels, all_preds)
    print("Confusion Matrix:\n", cm)

    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(all_labels),
                yticklabels=np.unique(all_labels), cbar=False)
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.title('Confusion Matrix')
    fig = plt.gcf()
    if writer:
        writer.add_figure("Evaluation/Confusion Matrix", fig)

    # Precision, Recall, F1-Score
    precision = precision_score(all_labels, all_preds, average='binary' if binary_classification else 'weighted')
    recall = recall_score(all_labels, all_preds, average='binary' if binary_classification else 'weighted')
    f1 = f1_score(all_labels, all_preds, average='binary' if binary_classification else 'weighted')
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")
    writer.add_scalar("Evaluation/Precision", precision)
    writer.add_scalar("Evaluation/Recall", recall)
    writer.add_scalar("Evaluation/F1-Score", f1)

    # AUC-ROC and ROC Curve (for binary classification)
    if binary_classification:
        all_probs = np.array(all_probs)
        if len(np.unique(all_labels)) == 2: 
            fpr, tpr, _ = roc_curve(all_labels, all_probs)
            auc_score = auc(fpr, tpr)
            print(f"AUC-ROC: {auc_score:.4f}")

            # Plot ROC Curve
            plt.figure()
            plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve (AUC = {auc_score:.4f})')
            plt.plot([0, 1], [0, 1], color='red', linestyle='--', lw=2)
            plt.xlabel('False Positive Rate')
            plt.ylabel('True Positive Rate')
            plt.title('Receiver Operating Characteristic (ROC) Curve')
            plt.legend(loc="lower right")
            plt.grid()
            fig = plt.gcf()
            if writer:
                writer.add_figure("Evaluation/ROC Curve", fig)
        else:
            print("AUC-ROC is not supported for multi-class tasks without modification.")

In [ ]:
evaluate_model_with_metrics(model, test_loader, binary_classification=True)

In [ ]:
def plot_confusion_roc(model, loader, binary_classification=True):
    model.eval()
    all_preds = []
    all_labels = []
    all_probs = [] 

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to('cuda'), labels.to('cuda')
            outputs = model(images)

            if hasattr(outputs, 'logits'):
                outputs = outputs.logits

            _, predicted = outputs.max(1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            # For ROC AUC
            if binary_classification:
                probabilities = F.softmax(outputs, dim=1)[:, 1].cpu().numpy()
                all_probs.extend(probabilities)

    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)

    # Confusion Matrix
    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(all_labels),
                yticklabels=np.unique(all_labels), cbar=False)
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.title('Confusion Matrix')
    plt.show()

    # ROC Curve (for binary classification)
    if binary_classification:
        fpr, tpr, _ = roc_curve(all_labels, all_probs)
        auc_score = auc(fpr, tpr)

        plt.figure()
        plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve (AUC = {auc_score:.4f})')
        plt.plot([0, 1], [0, 1], color='red', linestyle='--', lw=2)
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver Operating Characteristic (ROC) Curve')
        plt.legend(loc="lower right")
        plt.grid()
        plt.show()

plot_confusion_roc(model, test_loader, binary_classification=True)

# **COMPUTATIONAL EFFICIENCY METRIC**

In [ ]:
pip install fvcore

In [ ]:
import torch
import torch.nn as nn
from torchvision.models import convnext_base, ConvNeXt_Base_Weights
from torchinfo import summary
import time
from prettytable import PrettyTable
from fvcore.nn import FlopCountAnalysis, parameter_count_table


In [ ]:
image, label = train_dataset[0]  # Use the first image from the dataset
image = image.unsqueeze(0).to(device)
input_tensor = image
batch_size = 1

flops = FlopCountAnalysis(model, image)
total_flops = flops.total() / 1e9  # Convert to GFLOPs
params_table = parameter_count_table(model)
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

# Inference Time
start_time = time.time()
for _ in range(10):  # Run for 10 iterations
    with torch.no_grad():
        model(input_tensor)
end_time = time.time()
batch_inference_time = (end_time - start_time) / 10
per_image_inference_time = batch_inference_time / batch_size

# GPU Memory Usage
if device == 'cuda':
    gpu_allocated = torch.cuda.memory_allocated(device) / (1024 ** 3)
    gpu_reserved = torch.cuda.memory_reserved(device) / (1024 ** 3)
else:
    gpu_allocated = gpu_reserved = 0.0

# Throughput (FPS)
throughput = batch_size / batch_inference_time

from prettytable import PrettyTable
table = PrettyTable()
table.field_names = ["Metric", "Value"]
table.add_row(["FLOPs (GFLOPs)", f"{total_flops:.2f}"])
table.add_row(["Total Parameters", f"{total_params:,}"])
table.add_row(["Trainable Parameters", f"{trainable_params:,}"])
table.add_row(["Inference Time (Batch)", f"{batch_inference_time:.4f} seconds"])
table.add_row(["Inference Time (Per Image)", f"{per_image_inference_time:.4f} seconds"])
table.add_row(["Throughput (FPS)", f"{throughput:.2f} images/second"])
table.add_row(["GPU Memory Allocated", f"{gpu_allocated:.2f} GB"])
table.add_row(["GPU Memory Reserved", f"{gpu_reserved:.2f} GB"])

print(table)

# Log computational metrics to TensorBoard
writer.add_scalar("Efficiency/FLOPs (GFLOPs)", total_flops)
writer.add_scalar("Efficiency/Total Parameters", total_params)
writer.add_scalar("Efficiency/Inference Time (ms)", per_image_inference_time * 1000)
writer.add_scalar("Efficiency/Throughput (FPS)", throughput)


In [ ]:
writer.close()

# **SAVE MODEL & LOGS**

In [ ]:
path_50epochs = '/kaggle/working/model.pth'

torch.save(model.state_dict(), path_50epochs)

print("Models saved successfully to /kaggle/working/")

In [ ]:
!tar -czf tensorboard_logs.tar.gz runs